In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import sys
sys.path.append("/Users/alessiorussointroito/Documents/GitHub/Structural-Perturbation-Method")

#from SPM_fast import SPM
from SPM_cython import SPM

In [2]:
n_eigen = 10
k = 10

In [3]:
df = pd.read_csv(
    "/Users/alessiorussointroito/Downloads/Telegram Desktop/recommender-system-2020-challenge-polimi/data_train.csv")

le = LabelEncoder()
df['new_col'] = le.fit_transform(df.col)

row_size = len(df.row.unique())
col_size = len(le.classes_)
N_nodes = row_size + col_size

# Data Preprocessing
"""
Siccome un tipico problema di recsys è rappresentabile con una rete bipartita, dove in un set ci sono gli user e 
dall' altro gli items, in questa rete dobbiamo distinguere tutti i nodi. Di conseguenza supponiamo che gli items  
rappresentino i nodi che vanno da 0 a 25974, gli user invece vanno da 24895 a 24896 + |users|
"""

df['row'] = df.row + col_size

# Train test split
X_train, X_test, y_train, y_test = train_test_split(df.row, df.new_col, test_size=0.20, random_state=3)

# Make the urm symmetric, creating 2 urm and summing them. The two matrices represent upper triangle and down triangle
adj_down = sps.csr_matrix((np.ones(X_train.shape[0]), (X_train, y_train)), shape=(N_nodes, N_nodes))
adj_up = sps.csr_matrix((np.ones(X_train.shape[0]), (y_train, X_train)), shape=(N_nodes, N_nodes))
adj = adj_up + adj_down

test_indices = X_test.unique()
test_indices = np.sort(test_indices)
test = pd.DataFrame({'row': X_test, 'target': y_test})

targets = test.groupby(test.row)['target'].apply(lambda x: list(x))

In [17]:
spm = SPM(adj, target=test_indices , p=0.2, n_eigen=n_eigen)
rankings = spm.k_runs(k=k)

Remove seen : 100%|██████████| 10/10 [00:51<00:00,  5.16s/it]             


In [18]:
rankings

array([[ 6.56810030e-06,  1.42812142e-06,  2.91429428e-06, ...,
         4.54747351e-14, -5.68434189e-15, -2.22044605e-17],
       [ 6.82254693e-04,  3.41276879e-04,  1.93306171e-04, ...,
         9.82254278e-12,  9.09494702e-13, -3.80584453e-14],
       [ 2.05204812e-20,  4.70109245e-21,  1.43405104e-21, ...,
        -3.86139890e-20, -1.21691996e-20, -2.95082793e-24],
       ...,
       [ 8.21282345e-04, -1.10738382e-04,  5.33849263e-04, ...,
         0.00000000e+00, -3.63797881e-13, -1.42108547e-15],
       [ 1.08792197e-04,  4.99401695e-05,  3.96144543e-05, ...,
        -1.45519152e-12, -4.77484718e-13,  4.88498131e-15],
       [ 8.36585734e-04,  2.31056155e-04,  3.40714787e-04, ...,
                   -inf, -6.36646291e-13,  7.10542736e-15]])

In [19]:
# Evaluation

def precision(is_relevant, relevant_items):
    # is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
    return precision_score


def recall(is_relevant, relevant_items):
    # is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    recall_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
    return recall_score


def MAP(is_relevant, relevant_items):
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    map_score = np.sum(p_at_k) / np.min([len(relevant_items), len(is_relevant)])
    return map_score


In [20]:
n_users = len(test_indices)

cumulative_precision = 0.0
cumulative_recall = 0.0
cumulative_MAP = 0.0
num_eval = 0

at = 10

for i, user_id in enumerate(tqdm(test_indices)):
    relevant_items = targets[user_id]
    #recommended_items = rankings[user_id].argsort()[::-1][:at]
    recommended_items = rankings[i]

    # Filter Seen:
    # 1. Remove items already seen by the user
    seen_indices = sps.find(adj[user_id])[1]    # Con [1] Prendiamo solo le colonne d'interesse della matrice di adiacenza
    mask = np.zeros(N_nodes, dtype=bool)
    mask[seen_indices] = True

    # 2. Remove the other users since we want to recommend only the items for each user
    mask[col_size:] = True

    recommended_items[mask] = -np.inf

    # Recommend
    recommended_items = recommended_items.argsort()[::-1][:at]

    num_eval += 1

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    cumulative_precision += precision(is_relevant, relevant_items)
    cumulative_recall += recall(is_relevant, relevant_items)
    cumulative_MAP += MAP(is_relevant, relevant_items)

cumulative_precision /= num_eval
cumulative_recall /= num_eval
cumulative_MAP /= num_eval

print(f"SPM n_eigen = {n_eigen} , k = {k}  \n Precision = {cumulative_precision} \n Recall = {cumulative_recall} \n MAP = {cumulative_MAP}")

100%|██████████| 5644/5644 [00:16<00:00, 347.27it/s]

SPM n_eigen = 10 , k = 10  
 Precision = 0.009231041814316152 
 Recall = 0.009231041814316152 
 MAP = 0.011258762792265505


In [ ]:
SPM n_eigen = 10 , k = 10  
Precision = 0.009231041814316152 
Recall = 0.009231041814316152 
MAP = 0.011258762792265505

In [14]:
for i, user_id in enumerate(tqdm(test_indices)):    
    seen_indices = sps.find(adj[user_id])[1]    # Con [1] Prendiamo solo le colonne d'interesse della matrice di adiacenza
    mask = np.zeros(N_nodes, dtype=bool)
    mask[seen_indices] = True

    # 2. Remove the other users since we want to recommend only the items for each user
    mask[col_size:] = True
    rankings[i][mask] = -np.inf

100%|██████████| 5655/5655 [00:02<00:00, 2604.83it/s]


In [23]:
print(np.sort(rankings[1]))
print(rankings[1].argsort())

[      -inf       -inf       -inf ... 0.0006275  0.00077501 0.00135185]
[32842 28812 28811 ... 24605 24347 23063]


In [4]:
adj

<32843x32843 sparse matrix of type '<class 'numpy.float64'>'
	with 181228 stored elements in Compressed Sparse Row format>

In [5]:
adj.nnz

181228